In [1]:
import time
import datetime
import pandas as pd
#!pip install yfinance
import yfinance as yf
import numpy as np

In [26]:
historical_data = {}

ticker = "NVDA"
interval = "1d" 

data = yf.Ticker(ticker).history(period="10y", auto_adjust=False, interval=interval)
        
# Extract Adjusted Close and Date without hours
adj_close = np.array(data['Adj Close'])
time_horizon = np.array(pd.to_datetime(data.index).date)  # Convert to only year-month-day

# Store the date and adjusted close data
Data = {"Date": time_horizon, "Adj_close": adj_close}

df = pd.DataFrame(Data)

df.to_csv("test.csv")

In [29]:
test = yf.Ticker(ticker)
test

yfinance.Ticker object <NVDA>

In [30]:
test.option_chain()

KeyError: 'calls'

In [169]:
historical_data = {}

tickers = ("AAPL", "AMZN", "NFLX", "META", "GOOGL", "MSFT", "TSLA", "NVDA", "^GSPC")
intervals = ("1d", "1wk", "1mo")

for interval in intervals:
    period_data = {}
    for ticker in tickers:
        data = yf.Ticker(ticker).history(period="10y", auto_adjust=False, interval=interval)
        
        # Extract Adjusted Close and Date without hours
        adj_close = np.array(data['Adj Close'])
        time_horizon = np.array(pd.to_datetime(data.index).date)  # Convert to only year-month-day
        
        # Store the date and adjusted close data
        period_data[ticker] = {"Date": time_horizon, "Adj_close": adj_close}
        
    historical_data[interval] = period_data

In [170]:
# Dictionary to store DataFrames for each interval
dataframes = {}

# Iterate through each interval in the historical_data
for interval in historical_data:
    # Create an empty DataFrame to store data for all tickers in the given interval
    period_data = historical_data[interval]
    df_list = []
    for ticker in period_data:
        # Convert each ticker's data to a DataFrame
        ticker_df = pd.DataFrame(period_data[ticker]["Adj_close"], index=period_data[ticker]["Date"])
        ticker_df.columns = [f"{ticker}_Adj_Close"]  # Rename the column to include the ticker
        df_list.append(ticker_df)
    
    # Concatenate all ticker data for the given interval on the Date index
    combined_df = pd.concat(df_list, axis=1, join="outer",)
    # Store in the dataframes dictionary with the interval as the key
    dataframes[interval] = combined_df

# The resulting dataframes dictionary has 3 DataFrames, one for each interval (daily, weekly, monthly)
# You can access each DataFrame like this:
print("Daily Interval DataFrame:")
print(dataframes["1d"].info())

print("\nWeekly Interval DataFrame:")
print(dataframes["1wk"].info())

print("\nMonthly Interval DataFrame:")
print(dataframes["1mo"].info())

Daily Interval DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 2517 entries, 2014-11-10 to 2024-11-08
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   AAPL_Adj_Close   2517 non-null   float64
 1   AMZN_Adj_Close   2517 non-null   float64
 2   NFLX_Adj_Close   2517 non-null   float64
 3   META_Adj_Close   2517 non-null   float64
 4   GOOGL_Adj_Close  2517 non-null   float64
 5   MSFT_Adj_Close   2517 non-null   float64
 6   TSLA_Adj_Close   2517 non-null   float64
 7   NVDA_Adj_Close   2517 non-null   float64
 8   ^GSPC_Adj_Close  2517 non-null   float64
dtypes: float64(9)
memory usage: 196.6+ KB
None

Weekly Interval DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 522 entries, 2014-11-10 to 2024-11-04
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   AAPL_Adj_Close   522 non-null    float64
 1   AMZN_Adj_Close   5

In [171]:
for period in dataframes:
    dataframes[period].to_csv(f"Data\{period}_adj_close_price.csv", index=True)

<string>:2: SyntaxWarning: invalid escape sequence '\{'
<>:2: SyntaxWarning: invalid escape sequence '\{'
<string>:2: SyntaxWarning: invalid escape sequence '\{'
<>:2: SyntaxWarning: invalid escape sequence '\{'
C:\Users\lenovo\AppData\Local\Temp\ipykernel_19772\4260737570.py:2: SyntaxWarning: invalid escape sequence '\{'
  dataframes[period].to_csv(f"Data\{period}_adj_close_price.csv", index=True)


In [179]:
# Dictionary to store return DataFrames for each interval
returns_data = {}

# Iterate through each interval in historical_data
for interval, interval_data in historical_data.items():
    # Create a DataFrame to store returns for all tickers in the given interval
    interval_returns = {}
    
    for ticker, data in interval_data.items():
        # Convert the ticker data to a DataFrame
        ticker_df = pd.DataFrame(data)
        ticker_df.set_index("Date", inplace=True)
        
        # Calculate percentage returns from the previous period
        ticker_df['Return'] = ticker_df['Adj_close'].pct_change()
        #np.log(ticker_df['Adj_close'] / ticker_df['Adj_close'].shift(1))
        #ticker_df['Adj_close'].pct_change()
        
        # Store the returns in the period_returns dictionary
        interval_returns[ticker] = ticker_df['Return']
    
    # Concatenate all ticker returns for the interval into a single DataFrame
    returns_data[interval] = pd.DataFrame(interval_returns).dropna()

# Display the returns for each period
print("Daily Interval Returns:")
print(returns_data["1d"])

print("\nWeekly Interval Returns:")
print(returns_data["1wk"])

print("\nMonthly Interval Returns:")
print(returns_data["1mo"])

Daily Interval Returns:
                AAPL      AMZN      NFLX      META     GOOGL      MSFT  \
Date                                                                     
2014-11-11  0.007994  0.022615 -0.012188 -0.005200  0.005482 -0.000409   
2014-11-12  0.014130 -0.001603  0.003424  0.001474 -0.005416 -0.001841   
2014-11-13  0.014112  0.015955 -0.011593 -0.006290 -0.003242  0.017015   
2014-11-14  0.012054  0.035832  0.017475  0.008485 -0.002246 -0.000605   
2014-11-17 -0.001664 -0.014551 -0.012693 -0.008547 -0.015400 -0.002420   
...              ...       ...       ...       ...       ...       ...   
2024-11-04 -0.004038 -0.010862 -0.000780 -0.011425 -0.011968 -0.004654   
2024-11-05  0.006486  0.019001  0.011118  0.020957  0.002954  0.007345   
2024-11-06 -0.003267  0.038045  0.021338 -0.000664  0.039884  0.021193   
2024-11-07  0.021372  0.014293  0.020930  0.034350  0.024021  0.012495   
2024-11-08 -0.000725 -0.007094 -0.003603 -0.007895 -0.011757 -0.004819   

             

In [173]:
# Add bond yields to each interval
for interval in returns_data:
    t_bill = yf.Ticker("^IRX")
    t_bill_data = t_bill.history(period="10y", interval=interval, auto_adjust=False)
    try:
        if interval == "1d": 
            returns_data["1d"] = returns_data["1d"].drop(np.datetime64("2016-11-11"))
            returns_data[interval]["13W_TB_yields"] = np.array(t_bill_data["Adj Close"]/252)[1:]
        elif interval == "1wk":
            returns_data[interval]["13W_TB_yields"] = np.array(t_bill_data["Adj Close"]/52)[1:]
        elif interval == "1mo":
            returns_data[interval] = returns_data[interval][:113]
            returns_data[interval]["13W_TB_yields"] = np.array(t_bill_data["Adj Close"]/12)[1:114]
    except Exception as e:
        print(f"wrong at {interval}")
        print(t_bill_data["Adj Close"].index)
        print(returns_data[interval].index)
        pd.DataFrame(t_bill_data["Adj Close"]).to_csv(f"wrong_{interval}.csv", index =True)

In [180]:
t_bill = yf.Ticker("^IRX")
t_bill_data = t_bill.history(period="10y", interval="1d", auto_adjust=False)
t_bill_data.index[1:]

DatetimeIndex(['2014-11-11 00:00:00-06:00', '2014-11-12 00:00:00-06:00',
               '2014-11-13 00:00:00-06:00', '2014-11-14 00:00:00-06:00',
               '2014-11-17 00:00:00-06:00', '2014-11-18 00:00:00-06:00',
               '2014-11-19 00:00:00-06:00', '2014-11-20 00:00:00-06:00',
               '2014-11-21 00:00:00-06:00', '2014-11-24 00:00:00-06:00',
               ...
               '2024-10-28 00:00:00-05:00', '2024-10-29 00:00:00-05:00',
               '2024-10-30 00:00:00-05:00', '2024-10-31 00:00:00-05:00',
               '2024-11-01 00:00:00-05:00', '2024-11-04 00:00:00-06:00',
               '2024-11-05 00:00:00-06:00', '2024-11-06 00:00:00-06:00',
               '2024-11-07 00:00:00-06:00', '2024-11-08 00:00:00-06:00'],
              dtype='datetime64[ns, America/Chicago]', name='Date', length=2515, freq=None)

In [181]:
returns_data["1d"]

,AAPL,AMZN,NFLX,META,GOOGL,MSFT,TSLA,NVDA,^GSPC
Date,,,,,,,,,
2014-11-11,0.007994,0.022615,-0.012188,-0.005200,0.005482,-0.000409,0.037821,-0.011988,0.000697
2014-11-12,0.014130,-0.001603,0.003424,0.001474,-0.005416,-0.001841,-0.007886,-0.006572,-0.000701
2014-11-13,0.014112,0.015955,-0.011593,-0.006290,-0.003242,0.017015,0.010438,-0.005089,0.000530
2014-11-14,0.012054,0.035832,0.017475,0.008485,-0.002246,-0.000605,0.027731,0.012276,0.000240
2014-11-17,-0.001664,-0.014551,-0.012693,-0.008547,-0.015400,-0.002420,-0.018169,-0.004548,0.000735
...,...,...,...,...,...,...,...,...,...
2024-11-04,-0.004038,-0.010862,-0.000780,-0.011425,-0.011968,-0.004654,-0.024661,0.004801,-0.002812
2024-11-05,0.006486,0.019001,0.011118,0.020957,0.002954,0.007345,0.035414,0.028372,0.012266
2024-11-06,-0.003267,0.038045,0.021338,-0.000664,0.039884,0.021193,0.147510,0.040740,0.025296


In [176]:
for i in range(1, len(t_bill_data.index)):
    if t_bill_data.index[i] != returns_data["1d"].index[i-1]:
        print(f"{i}, { t_bill_data.index[i]}, {returns_data["1d"].index[i-1]}")

1, 2014-11-11 00:00:00-06:00, 2014-11-11
2, 2014-11-12 00:00:00-06:00, 2014-11-12
3, 2014-11-13 00:00:00-06:00, 2014-11-13
4, 2014-11-14 00:00:00-06:00, 2014-11-14
5, 2014-11-17 00:00:00-06:00, 2014-11-17
6, 2014-11-18 00:00:00-06:00, 2014-11-18
7, 2014-11-19 00:00:00-06:00, 2014-11-19
8, 2014-11-20 00:00:00-06:00, 2014-11-20
9, 2014-11-21 00:00:00-06:00, 2014-11-21
10, 2014-11-24 00:00:00-06:00, 2014-11-24
11, 2014-11-25 00:00:00-06:00, 2014-11-25
12, 2014-11-26 00:00:00-06:00, 2014-11-26
13, 2014-11-28 00:00:00-06:00, 2014-11-28
14, 2014-12-01 00:00:00-06:00, 2014-12-01
15, 2014-12-02 00:00:00-06:00, 2014-12-02
16, 2014-12-03 00:00:00-06:00, 2014-12-03
17, 2014-12-04 00:00:00-06:00, 2014-12-04
18, 2014-12-05 00:00:00-06:00, 2014-12-05
19, 2014-12-08 00:00:00-06:00, 2014-12-08
20, 2014-12-09 00:00:00-06:00, 2014-12-09
21, 2014-12-10 00:00:00-06:00, 2014-12-10
22, 2014-12-11 00:00:00-06:00, 2014-12-11
23, 2014-12-12 00:00:00-06:00, 2014-12-12
24, 2014-12-15 00:00:00-06:00, 2014-12-15
2

In [177]:
# Goes through each interval
for interval in returns_data:
    # Find the risk premium by taking the return of a stock and minus the yields according to each interval
    for stock in returns_data[interval].drop(columns=["13W_TB_yields"]):
        returns_data[interval][f"{stock}_minus_yields"] =  returns_data[interval][stock] -  returns_data[interval]["13W_TB_yields"]

In [178]:
for period in returns_data:
    returns_data[period].to_csv(f"Data\{period}_returns.csv", index=True)

<string>:2: SyntaxWarning: invalid escape sequence '\{'
<>:2: SyntaxWarning: invalid escape sequence '\{'
<string>:2: SyntaxWarning: invalid escape sequence '\{'
<>:2: SyntaxWarning: invalid escape sequence '\{'
C:\Users\lenovo\AppData\Local\Temp\ipykernel_19772\847485004.py:2: SyntaxWarning: invalid escape sequence '\{'
  returns_data[period].to_csv(f"Data\{period}_returns.csv", index=True)
